In [1]:
import pandas as pd
import clinicaldg.cxr.Constants as cxrConstants
import clinicaldg.cxr.process as cxrProcess

In [2]:
df_paths = cxrConstants.df_paths
df_paths

{'MIMIC': '/scratch/rc4499/thesis/data/mimic-cxr/clinicaldg/preprocessed.csv',
 'CXP': '/scratch/rc4499/thesis/data/chexpert/clinicaldg/preprocessed.csv',
 'NIH': '/scratch/rc4499/thesis/data/chestxray8/clinicaldg/preprocessed.csv',
 'PAD': '/scratch/rc4499/thesis/data/padchest/clinicaldg/preprocessed.csv'}

In [3]:
def get_frontal_prop(df):
    num_instances = len(df)
    num_pneumonia = df[df["frontal"] == 1]["frontal"].count()
    return num_pneumonia / num_instances

from os.path import exists
def img_exists(path):
    return exists(path)

dfs = {}
for env in cxrConstants.df_paths:
    func = cxrProcess.get_process_func(env)
    df_env = func(pd.read_csv(cxrConstants.df_paths[env]), only_frontal = False)
    # df_env["img_exists"] = df_env["path"].apply(img_exists)
    # df_env = df_env[df_env["img_exists"]]
    print(env, get_frontal_prop(df_env))
    dfs[env] = df_env

MIMIC 0.6452688154893861
CXP 0.8550445342681356
NIH 1.0
PAD 0.690928343045386


In [4]:
all_dfs = pd.concat([
    dfs["MIMIC"],
    dfs["CXP"],
    dfs["NIH"],
    dfs["PAD"]
])

all_dfs = all_dfs.fillna(0)

In [5]:
for col in disease_cols:
    print(all_dfs[col].value_counts())



NameError: name 'disease_cols' is not defined

In [5]:
group_cols = ["env", "frontal"]
# group_cols = "env"
disease_cols = ["No Finding", "Atelectasis", "Cardiomegaly", "Effusion", "Pneumonia", "Pneumothorax", "Consolidation", "Edema"]

# new_df = pd.DataFrame(data=[["NIH", False, 0, 0, 0, 0, 0, 0, 0, 0]], columns=group_cols + disease_cols)

by_view = all_dfs.groupby(group_cols).sum()[disease_cols]
# by_view = pd.concat([by_view, new_df])
# by_view.sort_values(["env", "frontal"], inplace=True)
by_view

No Finding  Atelectasis  Cardiomegaly  Effusion  Pneumonia  \
env   frontal                                                               
CXP   False        5419.0       3661.0        3617.0    9291.0     1364.0   
      True        17000.0      29795.0       23451.0   76963.0     4683.0   
MIMIC False       65019.0      16495.0       14810.0   18902.0     8534.0   
      True        84460.0      48895.0       41594.0   56625.0    16531.0   
NIH   True        60361.0      11559.0        2776.0   13317.0     1431.0   
PAD   False       14302.0       2446.0        4713.0    2523.0     2779.0   
      True        36127.0       5483.0        9065.0    6006.0     4894.0   

               Pneumothorax  Consolidation    Edema  
env   frontal                                        
CXP   False          1756.0         1801.0   2574.0  
      True          17700.0        13015.0  49717.0  
MIMIC False          2801.0         3039.0   7181.0  
      True          10027.0        11355.0  28845.0  
NIH   True           5302.0         4667.0   2303.0  
PAD   False           124.0          772.0    149.0  
      True            346.0         1561.0   1201.0

In [6]:
lateral_per_disease = all_dfs[~all_dfs["frontal"]].groupby("env").sum()[disease_cols]
lateral_per_disease.to_csv("c_lateral.csv")
lateral_per_disease

,No Finding,Atelectasis,Cardiomegaly,Effusion,Pneumonia,Pneumothorax,Consolidation,Edema
env,,,,,,,,
CXP,5419.0,3661.0,3617.0,9291.0,1364.0,1756.0,1801.0,2574.0
MIMIC,65019.0,16495.0,14810.0,18902.0,8534.0,2801.0,3039.0,7181.0
PAD,14302.0,2446.0,4713.0,2523.0,2779.0,124.0,772.0,149.0


In [7]:
total_per_disease = all_dfs.groupby("env").sum()[disease_cols]
total_per_disease.to_csv("c_total.csv")
total_per_disease

,No Finding,Atelectasis,Cardiomegaly,Effusion,Pneumonia,Pneumothorax,Consolidation,Edema
env,,,,,,,,
CXP,22419.0,33456.0,27068.0,86254.0,6047.0,19456.0,14816.0,52291.0
MIMIC,149479.0,65390.0,56404.0,75527.0,25065.0,12828.0,14394.0,36026.0
NIH,60361.0,11559.0,2776.0,13317.0,1431.0,5302.0,4667.0,2303.0
PAD,50429.0,7929.0,13778.0,8529.0,7673.0,470.0,2333.0,1350.0


In [48]:
proportion = lateral_per_disease / total_per_disease
proportion.to_csv("c_proportion.csv")
proportion

,No Finding,Atelectasis,Cardiomegaly,Effusion,Pneumonia,Pneumothorax,Consolidation,Edema
env,,,,,,,,
CXP,0.241715,0.109427,0.133626,0.107717,0.225566,0.090255,0.121558,0.049225
MIMIC,0.434971,0.252256,0.262570,0.250268,0.340475,0.218350,0.211130,0.199328
NIH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PAD,0.283607,0.308488,0.342067,0.295814,0.362179,0.263830,0.330904,0.110370


In [37]:
all_dfs["any1"] = (all_dfs[disease_cols]==1.0).any(axis="columns")
all_dfs

,subject_id,path,Sex,Age,env,frontal,study_id,No Finding,Atelectasis,Cardiomegaly,Effusion,Pneumonia,Pneumothorax,Consolidation,Edema,any1
0,10000032,/mimic-cxr/p10/p10000032/s50414267/02aa804e-bd...,F,40-60,MIMIC,True,50414267,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True
1,10000032,/mimic-cxr/p10/p10000032/s50414267/174413ec-4e...,F,40-60,MIMIC,False,50414267,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True
2,10000032,/mimic-cxr/p10/p10000032/s53189527/2a2277a9-b0...,F,40-60,MIMIC,True,53189527,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True
3,10000032,/mimic-cxr/p10/p10000032/s53189527/e084de3b-be...,F,40-60,MIMIC,False,53189527,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True
4,10000032,/mimic-cxr/p10/p10000032/s53911762/68b5c4b1-22...,F,40-60,MIMIC,True,53911762,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144489,112930952416074060371371014599496493673,/padchest/128401136192941452281465412169675154...,M,60-80,PAD,True,1284011361929414522814654121696751542351444145...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
144490,282743729971423358706056731890510600934,/padchest/128401136192941452209464657169675154...,F,60-80,PAD,True,1284011361929414522094646571696751542351444145...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True
144491,52648743308541843883453242716226652771,/padchest/128401136192941452208639063169675154...,M,40-60,PAD,True,1284011361929414522086390631696751542351444145...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
144492,228646130593152933811948996634154201216,/padchest/128401136192941452208410890169675154...,F,60-80,PAD,True,1284011361929414522084108901696751542351444145...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True


In [38]:
all_dfs["any1"].value_counts()

True     669722
False    186746
Name: any1, dtype: int64

In [54]:
by_view.loc[('MIMIC')]

,No Finding,Atelectasis,Cardiomegaly,Effusion,Pneumonia,Pneumothorax,Consolidation,Edema
frontal,,,,,,,,
False,65019,21466,27276,37282,37704,16117,10805,27301
True,84460,61246,70314,91426,60841,57290,24502,69450


In [47]:
cols = by_view.columns
by_view.loc[('NIH', False)] = [0, 0, 0, 0, 0, 0, 0, 0]

KeyError: 'cannot use a single bool to index into setitem'

In [42]:
by_view.loc[('CXP', False)]

No Finding        5439
Atelectasis       7867
Cardiomegaly      8274
Effusion         21467
Pneumonia         5109
Pneumothorax     10742
Consolidation    13682
Edema             8580
Name: (CXP, False), dtype: int64

In [30]:
pad = dfs["PAD"]
pad[pad["frontal"] == False]

,subject_id,path,Sex,Age,env,frontal,study_id,No Finding,Atelectasis,Cardiomegaly,Effusion,Pneumonia,Pneumothorax,Consolidation,Edema
1,313572750430997347502932654319389875966,/padchest/135803415504923515076821959678074435...,M,80-,PAD,False,135803415504923515076821959678074435083,0,0,0,0,0,0,0,0
4,50783093527901818115346441867348318648,/padchest/113855343774216031107737439268243531...,F,80-,PAD,False,113855343774216031107737439268243531979,0,0,0,0,0,0,0,0
5,93535126770783451980359712286922420997,/padchest/3137231742710829928-2476108022664036...,M,60-80,PAD,False,3137231742710829928-247610802266403640553,0,1,0,1,0,0,0,0
10,244685263100771469051214745085012030119,/padchest/105529804483623054726294337265160703...,F,60-80,PAD,False,105529804483623054726294337265160703666,0,0,0,0,0,0,0,0
13,139363266799141930129368850508503463058,/padchest/102185472428957491598043403159908631...,F,60-80,PAD,False,102185472428957491598043403159908631419,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144480,125425989422173300435392328459271503979,/padchest/128401136192941452468883831169675154...,M,80-,PAD,False,1284011361929414524688838311696751542351444145...,0,0,0,0,0,0,0,0
144482,62198729222724644096669167447191690937,/padchest/128401136192941452467838641169675154...,F,40-60,PAD,False,1284011361929414524678386411696751542351444145...,0,0,1,0,0,0,0,0
144486,178941107688363238116741298870229076976,/padchest/128401136192941452207984120169675154...,F,40-60,PAD,False,1284011361929414522079841201696751542351444145...,1,0,0,0,0,0,0,0
144487,178941107688363238116741298870229076976,/padchest/128401136192941452207984120169675154...,F,40-60,PAD,False,1284011361929414522079841201696751542351444145...,1,0,0,0,0,0,0,0


# Checking For Studies Without Frontals

In [6]:
def has_frontal(group):
    return group.any()

with_frontal = all_dfs.groupby("study_id")["frontal"].agg(has_frontal)

In [7]:
with_frontal

study_id
1                      True
10                     True
100                    True
1000                   True
10000                  True
                       ... 
patient64736/study1    True
patient64737/study1    True
patient64738/study1    True
patient64739/study1    True
patient64740/study1    True
Name: frontal, Length: 545012, dtype: bool

In [8]:
indices_without_frontal = with_frontal[~with_frontal].index
indices_without_frontal

Index(['101741800329932598958328421653105387931',
       '102536448805595370148169205121287134564_2',
       '104766698027297958280589609232122055294',
       '105566223538770174232067848507021687050_2',
       '106700336717044272880726422061056904209',
       '107453571276577049847802415416780956245',
       '10781146099800894534157212886940870380',
       '108032392908719484607354620855973001864_2',
       '109719662294069165640529331358364520285',
       '111828104960009130397249758380651308169',
       ...
       'patient07973/study1', 'patient09563/study2', 'patient11412/study3',
       'patient17678/study12', 'patient26100/study1', 'patient33890/study16',
       'patient51623/study6', 'patient53560/study1', 'patient60984/study1',
       'patient63296/study1'],
      dtype='object', name='study_id', length=10661)

In [11]:
images_without_frontal = all_dfs[all_dfs["study_id"].isin(indices_without_frontal)]

In [12]:
images_without_frontal

,subject_id,path,Sex,Age,env,frontal,study_id,No Finding,Atelectasis,Cardiomegaly,Effusion,Pneumonia,Pneumothorax,Consolidation,Edema
44,10001122,/mimic-cxr/p10/p10001122/s53447138/8039752c-2e...,F,60-80,MIMIC,False,53447138,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
45,10001122,/mimic-cxr/p10/p10001122/s53447138/832b57d8-3a...,F,60-80,MIMIC,False,53447138,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
130,10002013,/mimic-cxr/p10/p10002013/s55312734/227844e4-4c...,F,40-60,MIMIC,False,55312734,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
131,10002013,/mimic-cxr/p10/p10002013/s55312734/66ccec51-f0...,F,40-60,MIMIC,False,55312734,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
145,10002221,/mimic-cxr/p10/p10002221/s53781756/8b88f03a-20...,F,60-80,MIMIC,False,53781756,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144208,322226294479918533199030335357419871910,/padchest/216840111366964012819207061112010322...,F,40-60,PAD,False,216840111366964012819207061112010322141113829,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
144256,274291740032763798884772564498056823907,/padchest/216840111366964012819207061112010281...,F,80-,PAD,False,216840111366964012819207061112010281112908853,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
144270,120678344150550864453719616733452326527,/padchest/216840111366964012819207061112010307...,M,60-80,PAD,False,216840111366964012819207061112010307095251412,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
144463,113042320575397495544125056506517027249,/padchest/216840111366964012904401302362010328...,F,60-80,PAD,False,216840111366964012904401302362010328111720589,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
images_without_frontal.env.value_counts()

MIMIC    18733
PAD       1030
CXP         19
Name: env, dtype: int64

In [14]:
studies_without_frontal = all_dfs[all_dfs["study_id"].isin(indices_without_frontal)].groupby("env").nunique()["study_id"]
studies_without_frontal

env
CXP        16
MIMIC    9688
PAD       957
Name: study_id, dtype: int64

In [15]:
all_dfs

,subject_id,path,Sex,Age,env,frontal,study_id,No Finding,Atelectasis,Cardiomegaly,Effusion,Pneumonia,Pneumothorax,Consolidation,Edema
0,10000032,/mimic-cxr/p10/p10000032/s50414267/02aa804e-bd...,F,40-60,MIMIC,True,50414267,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10000032,/mimic-cxr/p10/p10000032/s50414267/174413ec-4e...,F,40-60,MIMIC,False,50414267,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10000032,/mimic-cxr/p10/p10000032/s53189527/2a2277a9-b0...,F,40-60,MIMIC,True,53189527,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,10000032,/mimic-cxr/p10/p10000032/s53189527/e084de3b-be...,F,40-60,MIMIC,False,53189527,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,10000032,/mimic-cxr/p10/p10000032/s53911762/68b5c4b1-22...,F,40-60,MIMIC,True,53911762,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144489,112930952416074060371371014599496493673,/padchest/128401136192941452281465412169675154...,M,60-80,PAD,True,1284011361929414522814654121696751542351444145...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
144490,282743729971423358706056731890510600934,/padchest/128401136192941452209464657169675154...,F,60-80,PAD,True,1284011361929414522094646571696751542351444145...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
144491,52648743308541843883453242716226652771,/padchest/128401136192941452208639063169675154...,M,40-60,PAD,True,1284011361929414522086390631696751542351444145...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
144492,228646130593152933811948996634154201216,/padchest/128401136192941452208410890169675154...,F,60-80,PAD,True,1284011361929414522084108901696751542351444145...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Checking Conditional Correlation / Dependence of Frontal / Lateral

If a study has a frontal, whats the proportion of time it also has a lateral...

In [20]:
def has_frontal(group):
    return group.any()

with_frontal = all_dfs.groupby("study_id")["frontal"].agg(has_frontal)
with_frontal

study_id
1                      True
10                     True
100                    True
1000                   True
10000                  True
                       ... 
patient64736/study1    True
patient64737/study1    True
patient64738/study1    True
patient64739/study1    True
patient64740/study1    True
Name: frontal, Length: 545012, dtype: bool

In [21]:
indices_with_frontal = with_frontal[with_frontal].index
indices_with_frontal

Index(['1', '10', '100', '1000', '10000', '10001',
       '100014625199913409730274754282179594842',
       '100015392272639845228940263464822056020',
       '100016903621660259605956393388617381387', '10002',
       ...
       'patient64731/study1', 'patient64732/study1', 'patient64733/study1',
       'patient64734/study1', 'patient64735/study1', 'patient64736/study1',
       'patient64737/study1', 'patient64738/study1', 'patient64739/study1',
       'patient64740/study1'],
      dtype='object', name='study_id', length=534351)

In [22]:
images_with_frontal = all_dfs[all_dfs["study_id"].isin(indices_with_frontal)]
images_with_frontal

,subject_id,path,Sex,Age,env,frontal,study_id,No Finding,Atelectasis,Cardiomegaly,Effusion,Pneumonia,Pneumothorax,Consolidation,Edema
0,10000032,/mimic-cxr/p10/p10000032/s50414267/02aa804e-bd...,F,40-60,MIMIC,True,50414267,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10000032,/mimic-cxr/p10/p10000032/s50414267/174413ec-4e...,F,40-60,MIMIC,False,50414267,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10000032,/mimic-cxr/p10/p10000032/s53189527/2a2277a9-b0...,F,40-60,MIMIC,True,53189527,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,10000032,/mimic-cxr/p10/p10000032/s53189527/e084de3b-be...,F,40-60,MIMIC,False,53189527,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,10000032,/mimic-cxr/p10/p10000032/s53911762/68b5c4b1-22...,F,40-60,MIMIC,True,53911762,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144489,112930952416074060371371014599496493673,/padchest/128401136192941452281465412169675154...,M,60-80,PAD,True,1284011361929414522814654121696751542351444145...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
144490,282743729971423358706056731890510600934,/padchest/128401136192941452209464657169675154...,F,60-80,PAD,True,1284011361929414522094646571696751542351444145...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
144491,52648743308541843883453242716226652771,/padchest/128401136192941452208639063169675154...,M,40-60,PAD,True,1284011361929414522086390631696751542351444145...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
144492,228646130593152933811948996634154201216,/padchest/128401136192941452208410890169675154...,F,60-80,PAD,True,1284011361929414522084108901696751542351444145...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
def has_lateral(group):
    return not group.all()

with_lateral = images_with_frontal.groupby("study_id")["frontal"].agg(has_lateral)
with_lateral

study_id
1                      False
10                     False
100                    False
1000                   False
10000                  False
                       ...  
patient64736/study1    False
patient64737/study1    False
patient64738/study1    False
patient64739/study1    False
patient64740/study1    False
Name: frontal, Length: 534351, dtype: bool

In [24]:
indices_with_lateral = with_lateral[with_lateral]
indices_with_lateral

study_id
100014625199913409730274754282179594842    True
100016903621660259605956393388617381387    True
100023894246484279964129120944560776912    True
100035238701184647172015593785663345624    True
100076021040072803090840594086153650249    True
                                           ... 
patient64618/study1                        True
patient64625/study1                        True
patient64640/study1                        True
patient64676/study1                        True
patient64705/study1                        True
Name: frontal, Length: 179545, dtype: bool

In [25]:
with_lateral.value_counts()

False    354806
True     179545
Name: frontal, dtype: int64

In [26]:
images_with_frontal

,subject_id,path,Sex,Age,env,frontal,study_id,No Finding,Atelectasis,Cardiomegaly,Effusion,Pneumonia,Pneumothorax,Consolidation,Edema
0,10000032,/mimic-cxr/p10/p10000032/s50414267/02aa804e-bd...,F,40-60,MIMIC,True,50414267,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10000032,/mimic-cxr/p10/p10000032/s50414267/174413ec-4e...,F,40-60,MIMIC,False,50414267,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10000032,/mimic-cxr/p10/p10000032/s53189527/2a2277a9-b0...,F,40-60,MIMIC,True,53189527,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,10000032,/mimic-cxr/p10/p10000032/s53189527/e084de3b-be...,F,40-60,MIMIC,False,53189527,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,10000032,/mimic-cxr/p10/p10000032/s53911762/68b5c4b1-22...,F,40-60,MIMIC,True,53911762,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144489,112930952416074060371371014599496493673,/padchest/128401136192941452281465412169675154...,M,60-80,PAD,True,1284011361929414522814654121696751542351444145...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
144490,282743729971423358706056731890510600934,/padchest/128401136192941452209464657169675154...,F,60-80,PAD,True,1284011361929414522094646571696751542351444145...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
144491,52648743308541843883453242716226652771,/padchest/128401136192941452208639063169675154...,M,40-60,PAD,True,1284011361929414522086390631696751542351444145...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
144492,228646130593152933811948996634154201216,/padchest/128401136192941452208410890169675154...,F,60-80,PAD,True,1284011361929414522084108901696751542351444145...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
images_with_lateral = images_with_frontal[images_with_frontal["study_id"].isin(indices_with_lateral.index)]
images_with_lateral

,subject_id,path,Sex,Age,env,frontal,study_id,No Finding,Atelectasis,Cardiomegaly,Effusion,Pneumonia,Pneumothorax,Consolidation,Edema
0,10000032,/mimic-cxr/p10/p10000032/s50414267/02aa804e-bd...,F,40-60,MIMIC,True,50414267,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10000032,/mimic-cxr/p10/p10000032/s50414267/174413ec-4e...,F,40-60,MIMIC,False,50414267,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10000032,/mimic-cxr/p10/p10000032/s53189527/2a2277a9-b0...,F,40-60,MIMIC,True,53189527,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,10000032,/mimic-cxr/p10/p10000032/s53189527/e084de3b-be...,F,40-60,MIMIC,False,53189527,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,10000764,/mimic-cxr/p10/p10000764/s57375967/096052b7-d2...,M,80-,MIMIC,True,57375967,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144485,178941107688363238116741298870229076976,/padchest/128401136192941452207984120169675154...,F,40-60,PAD,True,1284011361929414522079841201696751542351444145...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
144486,178941107688363238116741298870229076976,/padchest/128401136192941452207984120169675154...,F,40-60,PAD,False,1284011361929414522079841201696751542351444145...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
144487,178941107688363238116741298870229076976,/padchest/128401136192941452207984120169675154...,F,40-60,PAD,False,1284011361929414522079841201696751542351444145...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
144488,112930952416074060371371014599496493673,/padchest/128401136192941452281465412169675154...,M,60-80,PAD,False,1284011361929414522814654121696751542351444145...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
